In [1]:
import pandas as pd
df = pd.read_csv("/content/archive (7).zip")


In [2]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [9]:
x = scaler.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [11]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [12]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7837 - loss: 0.4576 - val_accuracy: 0.7468 - val_loss: 0.4844
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7839 - loss: 0.4438 - val_accuracy: 0.7403 - val_loss: 0.4832
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8119 - loss: 0.4239 - val_accuracy: 0.7403 - val_loss: 0.4827
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7899 - loss: 0.4323 - val_accuracy: 0.7338 - val_loss: 0.4817
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7957 - loss: 0.4423 - val_accuracy: 0.7273 - val_loss: 0.4821
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8049 - loss: 0.4154 - val_accuracy: 0.7338 - val_loss: 0.4808
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7944 - loss: 0.4554 - val_accuracy: 0.7403 - val_loss: 0.4814
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7878 - loss: 0.4290 - val_accuracy: 0.7338 - v

In [18]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [23]:
import kerastuner as kt

In [29]:
def build_model(hp):     #hp = hyperparameter
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [30]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [31]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [32]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [34]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [35]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7433 - loss: 0.5488 - val_accuracy: 0.7338 - val_loss: 0.5469
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7565 - loss: 0.5183 - val_accuracy: 0.7403 - val_loss: 0.5330
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7847 - loss: 0.4887 - val_accuracy: 0.7468 - val_loss: 0.5218
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7947 - loss: 0.4763 - val_accuracy: 0.7468 - val_loss: 0.5132
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7817 - loss: 0.4903 - val_accuracy: 0.7403 - val_loss: 0.5067
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7871 - loss: 0.4882 - val_accuracy: 0.7403 - val_loss: 0.5013
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7786 - loss: 0.4900 - val_accuracy: 0.7468 - val_loss: 0.4957
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7663 - loss: 0.4635 - val_accuracy: 0.75

### Now no. of neurons required for model

In [38]:
def  build_model(hp):
  model=Sequential()

  units = hp.Int('units',min_value=8,max_value=128,step=8)

  model.add(Dense(units=units),activation='relu',input_dims=8)
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [46]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [48]:
ser = tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [51]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [52]:
def build_model(hp):

  model=Sequential()

  model.add(Dense(72,activation='relu',input_dims=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model



In [53]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [55]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [56]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [58]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test)  )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8322 - loss: 0.3938 - val_accuracy: 0.7468 - val_loss: 0.4882
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8113 - loss: 0.4119 - val_accuracy: 0.7532 - val_loss: 0.4885
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8062 - loss: 0.4326 - val_accuracy: 0.7532 - val_loss: 0.4902
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8243 - loss: 0.3963 - val_accuracy: 0.7468 - val_loss: 0.4898
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8060 - loss: 0.4205 - val_accuracy: 0.7468 - val_loss: 0.4901
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8301 - loss: 0.4138 - val_accuracy: 0.7468 - val_loss: 0.4921
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8251 - loss: 0.4075 - val_accuracy: 0.7468 - val_loss: 0.4910
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8041 - loss: 0.4166 - val_accuracy: 0.74

In [62]:
def build_model(hp):
  model=Sequential()
  counter = 0
  units = hp.Int()
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:

      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation='relu',input_dim=8))

    else:
        model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                         activation = hp.choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
    counter += 1
    model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer = hp.choice('optimizer',values=['rmsprop','adam','sgd','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [65]:
tuner1 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [66]:
tuner1.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [68]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8297 - loss: 0.3772 - val_accuracy: 0.7597 - val_loss: 0.5040
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8389 - loss: 0.3616 - val_accuracy: 0.7597 - val_loss: 0.5014
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8160 - loss: 0.3939 - val_accuracy: 0.7597 - val_loss: 0.5013
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8024 - loss: 0.4093 - val_accuracy: 0.7532 - val_loss: 0.5016
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8279 - loss: 0.3513 - val_accuracy: 0.7597 - val_loss: 0.5031
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8324 - loss: 0.3649 - val_accuracy: 0.7597 - val_loss: 0.5028
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8262 - loss: 0.3698 - val_accuracy: 0.7532 - val_loss: 0.5022
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8494 - loss: 0.3576 - val_accuracy: 0.753